In [1]:
import lancedb
import pyarrow as pa
import pandas as pd
from datasets import load_dataset
from transformers import GPT2Tokenizer

/home/cord2108/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Para este ejercicio descargamos una base de datos de mensajes de texto en inglés.

In [2]:
dataset = load_dataset("chirunder/text_messages")

In [3]:
df = pd.DataFrame(dataset['train'])
df.rename(columns={'text': 'texto'}, inplace=True)
df = df.head(500)

Para los siguientes ejercicios, voy a crear una variable del numero de palabras en cada mensaje de texto.

In [4]:
df['n'] = df['texto'].apply(lambda x: len(str(x).split()))
df = df[['n', 'texto']]
df.head()

,n,texto
0,13,Top right I gained a little speed with the add...
1,14,They are heavier wheels though as are all the ...
2,9,Federally registering a trademark is more than...
3,21,I'll have to jog my memory from rooting a few ...
4,10,Unless you can afford to buy all new larger cl...


## Task 1:
A partir del dataframe df, crea df_tokenized (usando el Tokenizer de GPT2) con dos columnas pero con el texto tokenizado. Asegurate de que todos los embeddings tengan la misma longitud y los tokens sean enteros (todos enteros o todos doubles). 

In [40]:
# Cargar el tokenizer del modelo GPT2 de Huggingface
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Función para tokenizar y ajustar la longitud a 300 tokens
def tokenize_and_adjust_length(text):
    tokenized_text = tokenizer(text)["input_ids"]
    if len(tokenized_text) < 300:
        tokenized_text += [tokenizer.pad_token_id] * (300 - len(tokenized_text))
    else:
        tokenized_text = tokenized_text[:300]
    return tokenized_text

# Aplicar la tokenización al texto
df['tokenized_text'] = df['texto'].apply(tokenize_and_adjust_length)
print(df.columns)

df_tokenized = pd.DataFrame({'vector': df['tokenized_text'], 'texto': df['texto'], "n": df['n']})



Index(['n', 'texto', 'tokenized_text'], dtype='object')


## Task 2:
Mete el dataframe a una tabla en una base de datos de LanceDB.

In [41]:
# Nos conectamos a una base de datos local
db = lancedb.connect("./.lancedb")
# Creamos una tabla en la base de datos
db.create_table("tabla", df_tokenized)
db["tabla"].head()

pyarrow.Table
vector: fixed_size_list<item: float>[300]
  child 0, item: float
texto: string
n: int64
----
vector: [[[9126,826,314,8618,257,...,50256,50256,50256,50256,50256],[2990,389,20140,13666,996,...,50256,50256,50256,50256,50256],[37,5702,453,28336,257,...,50256,50256,50256,50256,50256],[40,1183,423,284,48342,...,50256,50256,50256,50256,50256],[28042,345,460,5368,284,...,50256,50256,50256,50256,50256]]]
texto: [["Top right I gained a little speed with the addition of IM heads.","They are heavier wheels though as are all the CV concave line from Vossen.","Federally registering a trademark is more than just aesthetics.","I'll have to jog my memory from rooting a few weeks ago, but it was a simple step from that point.","Unless you can afford to buy all new larger clothes!"]]
n: [[13,14,9,21,10]]

## Task 3:
Haz una query estilo SQL a la tabla de la base de datos. Quiero que escribas la query equivalente y pongas la explicación de lo que está haciendo la consulta. Hint: usa la columna "n". 

In [42]:
result_df = (
    db["tabla"].search()
    .where("n > 3")
    .select(["n", "vector"])
    .limit(11)
    .to_pandas()
)

# Mostrar los resultados
print(result_df)

     n                                             vector
0   13  [9126.0, 826.0, 314.0, 8618.0, 257.0, 1310.0, ...
1   14  [2990.0, 389.0, 20140.0, 13666.0, 996.0, 355.0...
2    9  [37.0, 5702.0, 453.0, 28336.0, 257.0, 16028.0,...
3   21  [40.0, 1183.0, 423.0, 284.0, 48342.0, 616.0, 4...
4   10  [28042.0, 345.0, 460.0, 5368.0, 284.0, 2822.0,...
5   12  [15784.0, 329.0, 257.0, 2524.0, 326.0, 318.0, ...
6   24  [2949.0, 14148.0, 4001.0, 475.0, 345.0, 761.0,...
7    4  [2990.0, 389.0, 16001.0, 19943.0, 13.0, 50256....
8   15  [1858.0, 3729.0, 389.0, 517.0, 6792.0, 5672.0,...
9   15  [1639.0, 3551.0, 880.0, 11.0, 290.0, 1944.0, 3...
10  13  [464.0, 1097.0, 4444.0, 510.0, 852.0, 2861.0, ...


- Query en SQL equivalente:
- Explicacion: 

Select n, tokenized_text
From tabla
Where n > 3

Estamos checando cuales son los textos con mas de tres palabras

## Task 4:
Inventa un mensaje de texto que tu podrías escribirle a un amigo. Tokenizalo y ponlo en el formato adecuado para hacer un vector query. Quiero que me regreses el mensaje más parecido al mensaje que inventaste (OJO: quiero el texto, no el embedding). HINT: Hay que decodear el resultado del query.

In [43]:
# Cargar el tokenizer del modelo GPT2 de Huggingface
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Establecer el token de relleno

# Mensaje de texto a tokenizar
mensaje = "Fuck you"

# Tokenizar el mensaje directamente con el tokenizer para obtener los 'input_ids'
mensaje_tokenizado = tokenizer(mensaje)["input_ids"]

# Longitud deseada del vector
longitud_deseada = 300

def ajustar_vector(input_ids, longitud_deseada):
    # Truncar si la lista es más larga que la longitud deseada
    output = input_ids[:longitud_deseada]
    # Rellenar con el ID del token de padding si es más corta
    pad_token_id = tokenizer.pad_token_id
    output += [pad_token_id] * (longitud_deseada - len(output))
    return output

# Ajustar el vector tokenizado a la longitud deseada
mensaje_tokenizado_ajustado = ajustar_vector(mensaje_tokenizado, longitud_deseada)

# Imprimir el resultado
print("Longitud del vector ajustado:", len(mensaje_tokenizado_ajustado))
print("Vector ajustado:", mensaje_tokenizado_ajustado)


Longitud del vector ajustado: 300
Vector ajustado: [34094, 345, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [50]:
result = (db["tabla"].search(mensaje_tokenizado_ajustado)
 .metric("L2") # Puede ser L2 o cosine
 .select(["vector"]) # Seleccionar la columna 'tokenized_text'
 .limit(1)
 .to_pandas())

vector = result['vector'][0]

original_string = tokenizer.decode(vector, skip_special_tokens=True)
print("String generado a partir del embedding:", original_string)

String generado a partir del embedding: Arizona?
